In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import * 

## 读入数据

2022.01.08

- 因为xarray会将读入nc文件含有units = days 的数据识别为日期，decode_times 需要关闭，从而读入dtype为float64的 天数，而非一个时间dtype

2022.01.13

- 因为heat_wave在4-5月难以达到阈值，因此计算绘制整体4-8月的heat_wave

2022.01.27

- 有进行MJJA时间段的平均态绘制的需求，对数据读入和子图部分做了很大修改

In [ ]:
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/mask_res/"
filename_obs  = "mask_sel_CN05.1_Tm_1961_2018_daily_025x025.nc"
filename_vr     = "mask_mean_t2m_98-17_VR.nc"
filename_rcm    = "mask_mean_t2m_98-17_RCM.nc"

# 平均气温
ds_or = {}
ds_or['obs'] = xr.open_dataset(dir_in + filename_obs)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var = {}
var['obs'] = ds_or['obs']['tm']#[:,  :, :]
var['vr'] = ds_or['vr']['t2m'] - 273.15
var['rcm'] = ds_or['rcm']['t2m'] - 273.15

# var['obs'] = var['obs'].reset_coords(names = 'lev', drop = True) # 去除掉obs中多余的lev coords

# change coords
var_list = ['obs', 'vr', 'rcm']
for i in var_list:
    rename_dict = dict(zip(var[i].coords.keys(), var['obs'].coords.keys()))
#     # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var[i] = var[i].rename(rename_dict)
    var[i]._coords = var['obs']._coords
    var[i] = var[i].rename(i)

In [ ]:
ds_in = {}
for mod_name in ['obs','vr', 'rcm']:
    ds_in[mod_name] = {}
    ds_in[mod_name]['mjja'] = xr.open_dataset\
        ("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/t2m/extreme/extreme_"+mod_name+"_5-8.nc", decode_times = False)
vars_names = list(ds_in[mod_name]['mjja'].variables)[3:]
vars_names

## 计算平均

In [ ]:
ds_mean = {}
for mod_name in ['obs','vr','rcm']:
    ds_mean[mod_name] = {}
    # 极端指数
    for season in ['mjja']:
        ds_mean[mod_name][season] = ds_in[mod_name][season].mean(dim = 'time')
        # 平均态
        daily_time_mjja = var['obs'].time.dt.month.isin([5,6,7,8])
        ds_mean[mod_name][season]['mean'] = var[mod_name].isel(time = daily_time_mjja).mean(dim = 'time')

lon = ds_mean['obs']['mjja']['mean'].lon.values
lat = ds_mean['obs']['mjja']['mean'].lat.values

## 绘图部分
2022.01.09

为了方便绘图，更换了版本，用来传入控制左边标题的参数 proplot 0.6.4-py_0 --> 0.9.5-pyhd8ed1ab_1

### 绘图准备部分
包含写为函数的地图的绘制，以等距的spacing norm

In [ ]:
## 地图绘制预备数据
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cmaps as cmaps

In [ ]:
def border_plot(axs):
    """
    进行行政区划的绘制，通过shapefilereader绘制存档的shp文件，需要传入axs，并逐个绘制
    比较消耗时间，调整完毕后最后添加边界的绘制
    """
    ##---- 直接绘图，从边界文件添加
    # for ax_ind in axs:
    # for line in borders:
    #     axs.plot(line[0::100], line[1::100], lw = 0.5, color='gray',transform=ccrs.Geodetic())
    #     axs.plot(line[0::10], line[1::10], lw = 0.4, color='black',transform=ccrs.Geodetic())
    ##---- 使用shp文件添加
        # shapefile数据下载的位置：
    # http://gaohr.win/site/blogs/2017/2017-04-18-GIS-basic-data-of-China.html
    world_border_shapefile = "/m2data2/yycheng/data_stage/CN-border/World/country.shp"
    river_border_shapefile =  "/raid52/yycheng/MPAS/REFERENCE/MODEL_CONSTANT/R1/" + "hyd1_4l.shp"
    southsea_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "southsea_island.shp"
    ninelines_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
    bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/m2data2/yycheng/data_stage/CN-border/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        # river     = shpreader.Reader(river_border_shapefile).geometries()
        river     = shpreader.Reader(river_border_shapefile, encoding = "gbk")
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
        # 绘制部分的shapefile
        for region in river.records():
            if (region.attributes['NAME'] in ['黄河','长江']):
                # print("----- draw river! -----")
                # 此处需要使用 [] 让region.geometry可以迭代
                ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)


### 绘图测试部分
因为子图过多，此处测试色板 norm ticks等要素

In [ ]:
vars_names

In [ ]:
# ticks_am = np.linspace(5, 20, int(15 / 0.5 + 1)) # daily_temperature_range
# ticks_jja = np.linspace(5, 20, int(15 / 0.5 + 1)) # daily_temperature_range

ticks_jja = np.linspace(1,5,17) # daily_temperature_range_variability
ticks_am = np.linspace(1,5,17) # daily_temperature_range_variability

ticks_am = np.linspace(0, 10, 21) # heat_wave_total_length
ticks_jja = np.concatenate( (np.linspace(0, 30, 31),[40,50,60,70,80]), axis = 0) # heat_wave_total_length

# ticks_am = np.linspace(0,10,21) # heat_wave_max_length
# ticks_jja = np.concatenate( (np.linspace(0, 20, 21),[25,30,35,40]), axis = 0) # heat_wave_max_length

# cmap = cmaps.BkBlAqGrYeOrReViWh200  # daily_temperature_range
# cmap = cmaps.WhViBlGrYeOrReWh # heat_wave_frequency
cmap = cmaps.WhViBlGrYeOrReWh # heat_wave_total_length
# cmap = cmaps.WhViBlGrYeOrReWh  # hot_spell_max_length

fig, axs = pplt.subplots(nrows = 2,ncols = 3)
var = 'heat_wave_total_length'

mcontourf_am = axs[0].contourf(ds_mean['obs']['amjja'][var].values, cmap = cmap, levels = ticks_jja, norm = 'segmented')
mcontourf_am = axs[1].contourf(ds_mean['vr']['amjja'][var].values, cmap = cmap, levels = ticks_jja, norm = 'segmented')
mcontourf_am = axs[2].contourf(ds_mean['rcm']['amjja'][var].values, cmap = cmap, levels = ticks_jja, norm = 'segmented')

mcontourf_jja = axs[3].contourf(ds_mean['obs']['jja'][var].values, cmap = cmap, levels = ticks_jja, norm = 'segmented')
mcontourf_jja = axs[4].contourf(ds_mean['vr']['jja'][var].values, cmap = cmap, levels = ticks_jja, norm = 'segmented')
mcontourf_jja = axs[5].contourf(ds_mean['rcm']['jja'][var].values, cmap = cmap, levels = ticks_jja, norm = 'segmented')

axs.format(title = var,
leftlabels=(('test'),('test')),
leftlabelsize = 10,)

axs[2].colorbar(mcontourf_am, ticks = ticks_am, norm = 'segmented', length = 0.9, title=var, labelsize = 12)
axs[5].colorbar(mcontourf_jja, ticks = ticks_jja, norm = 'segmented', length = 0.9, title=var, labelsize = 8)

### 完整绘图部分

In [ ]:
vars_names = ['daily_temperature_range',
'daily_temperature_range_variability',
'heat_wave_total_length',
'heat_wave_max_length',]

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
import collections
# ----- get filter vars coords-----

#----- create plot -----
shapearray = [ [1,2,3,0,0,0],\
               [4,5,6,7,8,9],\
               [10,11,12,13,14,15]]
fig, axs = plot.subplots(shapearray, proj=('cyl'), share = True)
m_contour_list = [] # 用于保存contour设置，后续设置colorbar使用

#----- 添加海洋以及行政区划 -----
border_plot(axs)

def nested_dict():
    return collections.defaultdict(nested_dict)
mcontourf_dict = nested_dict()
#----- colorbar ticks 统一设置 -----
cmap_dict = {}

cmap_dict['mean'] = cmaps.t2m_29lev
cmap_dict['mean_diff'] = cmaps.ncl_default
cmap_dict['daily_temperature_range'] = cmaps.BkBlAqGrYeOrReViWh200
cmap_dict['daily_temperature_range_variability'] = cmaps.WhViBlGrYeOrReWh
cmap_dict['heat_wave_total_length'] = cmaps.WhViBlGrYeOrReWh
cmap_dict['heat_wave_max_length'] = cmaps.WhViBlGrYeOrReWh

ticks_dict = nested_dict()

ticks_dict['mjja']['mean']                    = np.linspace(3,30,int(27/1) + 1)
# ticks_dict['mjja']['mean_diff']               = np.concatenate( ([-12],[-10],[-8],np.linspace(-5, 5, 21),[8],[10],[12]), axis = 0 )
ticks_dict['mjja']['mean_diff']               = np.linspace(-8, 8, 33)
ticks_dict['mjja']['daily_temperature_range'] = np.linspace(5, 20, int(15 / 0.5 + 1))
ticks_dict['mjja']['daily_temperature_range_variability']     = np.linspace(1,4,31) # daily_temperature_range_variability
ticks_dict['mjja']['heat_wave_total_length']  =np.concatenate( (np.linspace(0, 30, 31),[40,50,60,70,80]), axis = 0) # heat_wave_total_length
ticks_dict['mjja']['heat_wave_max_length']    =  np.concatenate( (np.linspace(0, 20, 21),[25,30,35,40]), axis = 0) # heat_wave_max_length

ticks_dict['mjja']['heat_wave_total_length']   = np.linspace(0, 10, 21) # heat_wave_total_length
ticks_dict['mjja']['heat_wave_total_length']  =np.concatenate( (np.linspace(0, 30, 31),[40,50,60,70,80]), axis = 0) # heat_wave_total_length
ticks_dict['mjja']['heat_wave_max_length']     = np.linspace(0,10,21) # heat_wave_max_length
ticks_dict['mjja']['heat_wave_max_length']    =  np.concatenate( (np.linspace(0, 20, 21),[25,30,35,40]), axis = 0) # heat_wave_max_length

# ----- plots setting -----
plot_ind = 0
# 绘制平均态 & 平均态做差

mcontourf_temp = axs[0].contourf(lon, lat, ds_mean['obs']['mjja']['mean'].values,\
cmap=cmap_dict['mean'], levels = ticks_dict['mjja']['mean'], norm = 'segmented')
mcontourf_dict['mjja']['obs']['mean'] = mcontourf_temp # 合并所有绘图obj到一个deep dict之中
mcontourf_temp_diff = axs[1].contourf(lon, lat, ds_mean['vr']['mjja']['mean'].values - ds_mean['obs']['mjja']['mean'].values,\
cmap=cmap_dict['mean_diff'], levels = ticks_dict['mjja']['mean_diff'], norm = 'div', extend = 'both')
mcontourf_temp_diff = axs[2].contourf(lon, lat, ds_mean['rcm']['mjja']['mean'].values - ds_mean['obs']['mjja']['mean'].values,\
cmap=cmap_dict['mean_diff'], levels = ticks_dict['mjja']['mean_diff'], norm = 'div', extend = 'both')

plot_ind = 3
for var in vars_names:    
    for season in ['mjja']:
        for mod_name in ['obs','vr','rcm']:
            # 使用了均分的norm，处理不等距的ticks
            mcontourf_temp = axs[plot_ind].contourf(lon, lat, ds_mean[mod_name][season][var].values,\
            cmap=cmap_dict[var], levels = ticks_dict[season][var], norm = 'segmented')
            mcontourf_dict[season][mod_name][var] = mcontourf_temp # 合并所有绘图obj到一个deep dict之中
            plot_ind = plot_ind + 1

#----- add color bar-----
color_bar_title = {}
color_bar_title['mean'] = 'daily mean temperature ' + r"$[^\circ C$]"
color_bar_title['mean_diff'] = 'biases ' + r"$[^\circ C$]"
color_bar_title['daily_temperature_range'] = 'daily temperature range ' + r"$[^\circ C$]"
color_bar_title['daily_temperature_range_variability']   = 'range variability ' + r"$[^\circ C$]"
color_bar_title['heat_wave_total_length']    = 'heat wave total length ' + "[days]"
color_bar_title['heat_wave_max_length']    = 'heat wave max length ' + "[days]"

# 平均态 colorbar
axs[2].colorbar(mcontourf_dict['mjja']['obs']['mean'], loc='r', width=0.1, extend = 'both',length = 0.96,\
ticklabelsize=5,labelsize = 7,ticks=ticks_dict['mjja']['mean'][::2], title=color_bar_title['mean'])
axs[2].colorbar(mcontourf_temp_diff, loc='r', width=0.1, extend = 'both',length = 0.96,\
ticklabelsize=5,labelsize = 7,ticks=ticks_dict['mjja']['mean_diff'][::2], title=color_bar_title['mean_diff'])
# 极端指数 colorbar
plot_ind = 5
# colorbar_locations = [(1,2), (1,5), (2,2), (2,5)]
for var in vars_names:
    axs[plot_ind].colorbar(mcontourf_dict['mjja']['obs'][var], loc='r', width=0.1, extend = 'both',length = 0.96,\
    ticklabelsize=5,labelsize = 7,ticks=ticks_dict['mjja'][var][::2], title=color_bar_title[var])
    plot_ind = plot_ind + 3

# ----- format setting -----
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = True,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
# cartopyautoextent = True, 
# borderslinewidth=.5,
labels = False,
longrid  = True,
latgrid  = True,
#-----GEO axis-----
lonlim=(70, 140), latlim=(15, 55),
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,142,10),
latlocator = np.arange(15,55+2,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(15,55+2,2),
#-----line label-----
# linewidth = 0.5,
# suptitle="3000km-2000km bandpass 500hPa height(1998-06 timemean)",
# suptitle="precipitation",
# toplabels=(' \nOBS', 'AM\nVR', ' \nRCM', ' \nOBS', 'JJA\nVR', ' \nRCM'),
# leftlabels=('daily temperature range','daily temperature range variability','heat wave total length','heat wave max length'),
leftlabelsize = 6.,
rc_kw = {'leftlabel.rotation':90.},
)

# ----- add axses title and labels setting -----
axs.format(titleweight = 'bold')
axs[0,0].format(title = "OBS")
axs[0,1].format(title = "mean\nVR")
axs[0,2].format(title = "RCM")

axs[1,0].format(title = "OBS")
axs[1,1].format(title = "daily temperature range\nVR")
axs[1,2].format(title = "RCM")

axs[1,3].format(title = "OBS")
axs[1,4].format(title = "daily temperature range variability\nVR")
axs[1,5].format(title = "RCM")

axs[2,1].format(title = "heat wave total length")
axs[2,4].format(title = "heat wave max length")

# 合并子图之后控制边界的labels绘制
axs[:-1,0].format(labels = True, lonlabels = False, latlabels = True)
axs[2,1:].format(labels = True, lonlabels = True, latlabels = False)
axs[-1,0].format(labels = True, lonlabels = True, latlabels = True)

# # 删除子图控制AMJJA的热浪的子图 hide axes
# axs[13].format(title = "AMJJA", titleweight = 'bold')
# for i in range(15,18):
#     fig.delaxes(axs[i])
# for i in range(21,24):
#     fig.delaxes(axs[i])

#----- save figure -----
fig.patch.set_facecolor('white')
plt.savefig('./output_pic/t2m_meanstates_2022.01.27.png', dpi=600, facecolor= "white")